In [ ]:
#Adding dependecies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress

In [ ]:
#opening covid data
covid_data=pd.read_csv("data/worldometer_coronavirus_daily_data.csv")

<h1>Analysis of Covid situation in UK</h1>

In [ ]:
#selecting data only relevant to UK
uk_covid=covid_data[covid_data["country"]=="UK"]
uk_covid.to_csv("output/UK_covid_data.csv", index=False, header=True)
uk_covid.head()

<h3>Daily new Covid cases in UK</h3>

In [ ]:
#selecting data from every 30th row to display monthly situation
uk_covid2 = uk_covid[uk_covid.index % 30== 0]
date=uk_covid2["date"]
daily_new_cases=uk_covid2["daily_new_cases"]
#plotting line graph of daily cases
plt.plot(date, daily_new_cases)
plt.title("Daily New Covid Cases in UK from March 2020 to November 2021")
plt.xticks(rotation=45)
plt.savefig("images/UK_daily_cases.png")
plt.show()

<h3>Daily Deaths in UK</h3>

In [ ]:
#plotting line graph of daily deaths
daily_new_deaths=uk_covid2["daily_new_deaths"]
date=uk_covid2["date"]
plt.plot(date, daily_new_deaths)
plt.title("Daily New Deaths in UK from March 2020 to November 2021")
plt.xticks(rotation=45)
plt.savefig("images/UK_daily_deaths.png")
plt.show()

<h3>Daily new cases vs daily deaths in UK</h3>

In [ ]:
#plotting scatter plot of daily new cases vs new deaths
plt.scatter(daily_new_cases,daily_new_deaths, marker="o", facecolors="blue", alpha=0.75)
plt.xlabel("Daily New Cases")
plt.ylabel("Daily New Deaths")
plt.title("UK Daily New Cases vs Deaths")

In [ ]:
#adding linear regression to the scatter plot
(slope, intercept, rvalue, pvalue, stderr) = linregress(daily_new_cases, daily_new_deaths)
regress_values=daily_new_cases*slope+intercept
line_eq= "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(daily_new_cases,daily_new_deaths, marker="o", facecolors="blue", alpha=0.75)
plt.plot(daily_new_cases,regress_values,"r-")
plt.annotate(line_eq,(27000,600),fontsize=15, color="red")
plt.xlabel("Daily New cases")
plt.ylabel("Daily new deaths")
print(f"The r-squared is: {rvalue**2}")
plt.title("UK Daily New Cases vs Deaths")
plt.savefig("images/UK_daily_cases_vs_deaths.png")

R-squared value reall low indicating not strong correlation between Daily New Cases and Daily Deaths in UK

<h2> Looking into number of cases vs vaccination rate</h2>

In [ ]:
#importing vaccination data set
vaccine_data=pd.read_csv("data/country_vaccinations.csv")
#setting date to the format to enable merging
vaccine_data["date"]=pd.to_datetime(vaccine_data.date)
covid_data["date"]=pd.to_datetime(covid_data.date)

In [ ]:
#vaccination dataset splits UK into constituent countries
#extracting data for each country and merging in order to make the datasets correlating
eng_vaccine=vaccine_data[vaccine_data["country"]=="England"]
eng_vaccine=eng_vaccine[["country","date","daily_vaccinations", "people_fully_vaccinated", "people_vaccinated_per_hundred"]]
scot_vaccine=vaccine_data[vaccine_data["country"]=="Scotland"]
scot_vaccine=scot_vaccine[["country","date","daily_vaccinations","people_fully_vaccinated", "people_vaccinated_per_hundred"]]
wal_vaccine=vaccine_data[vaccine_data["country"]=="Wales"]
wal_vaccine=wal_vaccine[["country","date","daily_vaccinations","people_fully_vaccinated", "people_vaccinated_per_hundred"]]
irl_vaccine=vaccine_data[vaccine_data["country"]=="Northern Ireland"]
irl_vaccine=irl_vaccine[["country","date","daily_vaccinations","people_fully_vaccinated", "people_vaccinated_per_hundred"]]
uk_merged1=pd.merge(eng_vaccine,scot_vaccine,on="date",how="outer",suffixes=('1', '2'))
uk_merged2=pd.merge(uk_merged1,wal_vaccine,on="date",how="outer",suffixes=('2', '3'))
uk_merged3=pd.merge(uk_merged2,irl_vaccine,on="date",how="outer",suffixes=('3', '4'))
uk_merged3=uk_merged3.fillna(value=0)
uk_merged3=uk_merged3.sort_values(by=["date"])
uk_merged3.head()

In [ ]:
#calculating the sum of daily vaccinations in each constituent countries
#to find out th daily vaccination across the whole UK
uk_daily_vacc=[]
for row in uk_merged3:
    daily_sum=uk_merged3["daily_vaccinations1"]+uk_merged3["daily_vaccinations2"]+uk_merged3["daily_vaccinations3"]+uk_merged3["daily_vaccinations4"]
    uk_daily_vacc.append(daily_sum)
uk_daily_vacc[0]

In [ ]:
uk_fully_vacc=[]
for row in uk_merged3:
    fully_vacc_sum=uk_merged3["people_fully_vaccinated1"]+uk_merged3["people_fully_vaccinated2"]+uk_merged3["people_fully_vaccinated3"]+uk_merged3["people_fully_vaccinated4"]
    uk_fully_vacc.append(fully_vacc_sum)
uk_fully_vacc[0]

In [ ]:
#creating new data frame holding combined data for England, Wales, Scotland and Northern Ireland
#calling it the UK dataset
uk_vaccination_df=pd.DataFrame({"country":"UK",
                                "date":uk_merged3["date"],
                               "daily vaccinations":uk_daily_vacc[0],
                               "people fully vaccinated":uk_fully_vacc[0]})
uk_vaccination_df.to_csv("output/UK_vaccination_data.csv", index=False, header=True)
uk_vaccination_df.head()

In [ ]:
final_vacc_number=uk_vaccination_df[["country","people fully vaccinated"]].drop_duplicates(subset=['country'], keep="last")
final_vacc_number

In [ ]:
#extracting UK data from the covid dataset
uk_covid=covid_data[covid_data["country"]=="UK"]
uk_covid=uk_covid[["date","daily_new_cases"]]

In [ ]:
#merging two datasets to display full dataframe of vaccination and covid cases across UK
uk_combined=pd.merge(uk_vaccination_df,uk_covid, on="date")
uk_combined=uk_combined.dropna()
uk_combined.head()

In [ ]:
#scatter plot of daily cases vs daily vaccination in UK
uk_combined_vacc=uk_combined["daily vaccinations"]
uk_combined_cases=uk_combined["daily_new_cases"]
plt.scatter(uk_combined_cases,uk_combined_vacc, marker="o", facecolors="blue", alpha=0.75)
plt.xlabel("Daily New Cases")
plt.ylabel("Daily Vaccinations")
plt.title("Daily new cases vs daily vaccination in UK")

In [ ]:
#adding linear regression to the scatter plot
(slope, intercept, rvalue, pvalue, stderr) = linregress(uk_combined_vacc, uk_combined_cases)
regress_values=uk_combined_vacc*slope+intercept
line_eq= "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
print(f"The r-squared is: {rvalue**2}")
plt.scatter(uk_combined_vacc,uk_combined_cases, marker="o", facecolors="blue", alpha=0.75)
plt.plot(uk_combined_vacc,regress_values,"r-")
plt.annotate(line_eq,(500,10000),fontsize=15, color="red")
plt.xlabel("Daily Vaccinations")
plt.ylabel("Daily New Cases")
plt.title("Daily daily vaccination vs new cases in UK")
plt.savefig("images/uk_daily_cases_vs_vaccination.png")
plt.show

In [ ]:
#selecting every 7th row from the dataset to collect weekly data
uk_combined_timeframe = uk_combined[uk_combined.index % 7== 0]

In [ ]:
#creating scatter plot to display weekly vaccination progress in UK
time_uk=uk_combined_timeframe["date"]
daily_Vacc=uk_combined_timeframe["daily vaccinations"]
plt.scatter(time_uk, daily_Vacc, marker="o", facecolors="red", edgecolors="black",alpha=0.75)
plt.xticks(rotation=45)
plt.ylabel("Vaccination")
plt.title("Vaccination over time in UK")
plt.savefig("images/uk_vaccination_over_time.png")